# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Load the csv 
output_file = "../cities_output_student.csv"

# Create DataFrame for output data file from cities/weather (Part I)
cities_df = pd.read_csv(r'../cities_output_student.csv')
cities_df.head()


,Cities,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,washington,38.612744,-91.068242,74.35,26,100,3.20
1,barrow,83.497291,-156.870956,42.82,87,90,6.91
2,leningradskiy,85.179396,178.633670,35.89,98,100,4.72
3,tuktoyaktuk,87.348126,-133.156524,48.20,87,90,8.05
4,albany,-80.791137,120.641077,81.79,60,5,2.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store lat and lng as locations
locations = cities_df[['Latitude','Longitude']]

# Store humidity as weight
weight = cities_df['Humidity'].astype(float)

# Plot Heatmap
fig =gmaps.figure()

# Creat heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Creat new DataFrame fitting ideal conditions
vacation_spot_df = cities_df.loc[(cities_df['Temperature'] > 65) & (cities_df['Temperature'] < 85) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] < 30)].dropna()
vacation_spot_df

,Cities,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
4,albany,-80.791137,120.641077,81.79,60,5,2.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Store variable named hotel_df
hotel_df = vacation_spot_df

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

# Define parameters
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}

# Loop through API
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
        
hotel_df

Missing result.... Skipping.


,Cities,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
4,albany,-80.791137,120.641077,81.79,60,5,2.95,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
